In [2]:
!pip install transformers sentence-transformers gradio torch --quiet

In [3]:
!pip install fastapi uvicorn nest_asyncio pyngrok --quiet

In [4]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import gradio as gr

In [5]:
import gradio as gr
from pyngrok import ngrok

In [6]:
import nest_asyncio
import threading
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

In [7]:
# Sentiment Analysis
sentiment_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
LABEL_MAP = {"LABEL_0": "NEGATIVE", "LABEL_1": "NEUTRAL", "LABEL_2": "POSITIVE"}

# Sentence Similarity
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
def analyze_sentiment(text):


    if not text.strip():
        return "Please enter some text!"

    results = sentiment_model(text)
    #results = [{"label": "LABEL_2", "score": 0.987},{"label": "LABEL_1", "score": 0.012}]

    mapped = []
    for r in results:
        raw_label = r["label"]
        readable_label = LABEL_MAP[raw_label]
        score = r["score"]
        mapped.append({
           "label": readable_label,
           "score": score
    })
    #mapped = [{"label": "POSITIVE", "score": 0.987},{"label": "NEUTRAL", "score": 0.012}]


    lines = []
    for item in mapped:
        label = item["label"]
        score = item["score"]


        formatted_line = f"{label} → {score:.3f}"


        lines.append(formatted_line)


        final_text = "\n".join(lines) #تبدیل به یک رشته چند خطی

        #lines = [
                #"POSITIVE → 0.987",
                #"NEUTRAL → 0.012",
                #"NEGATIVE → 0.001"]


    return final_text



In [9]:
def check_similarity(source_sentence, other_sentences):

    if not source_sentence.strip() or not other_sentences.strip():
        return "Please enter both source and other sentences!"

    others = []
    for s in other_sentences.split(","):
        cleaned = s.strip()
        if cleaned:   # یعنی خالی نبود
            others.append(cleaned)


    emb1 = similarity_model.encode(source_sentence, convert_to_tensor=True)
    emb2 = similarity_model.encode(others, convert_to_tensor=True)

    similarities = util.cos_sim(emb1, emb2)

    result_text = ""
    for sentence, score in zip(others, similarities[0]):
        result_text += f"{sentence} → {score.item():.3f}\n"

    return result_text


In [ ]:
ngrok.kill()
ngrok.set_auth_token("Your Token")
gr_url = ngrok.connect(7865)
print("Gradio Public URL:", gr_url)

with gr.Blocks() as demo:
    gr.Markdown("## AI Tools: Sentiment & Sentence Similarity")

    with gr.Tab("Sentiment Analysis"):
        text_input = gr.Textbox(label="Enter text", lines=5)
        sentiment_output = gr.Textbox(label="Sentiment Result", lines=5)
        gr.Button("Analyze Sentiment").click(analyze_sentiment, text_input, sentiment_output)

    with gr.Tab("Sentence Similarity"):
        source_input = gr.Textbox(label="Source Sentence")
        other_input = gr.Textbox(label="Other Sentences (comma separated)", lines=5)
        similarity_output = gr.Textbox(label="Similarity Result", lines=5)
        gr.Button("Check Similarity").click(check_similarity, [source_input, other_input], similarity_output)

demo.launch(server_name="0.0.0.0", server_port=7865,share=True)


Gradio Public URL: NgrokTunnel: "https://grenadierly-nonspinose-mollie.ngrok-free.dev" -> "http://localhost:7865"
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://70f008ffb705f6caf2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
ngrok.kill()
app = FastAPI()

class SentimentRequest(BaseModel):
    text: str

class SimilarityRequest(BaseModel):
    source: str
    others: str

@app.post("/sentiment")
def api_sentiment(req: SentimentRequest):
    return {"result": analyze_sentiment(req.text)}

@app.post("/similarity")
def api_similarity(req: SimilarityRequest):
    return {"result": check_similarity(req.source, req.others)}


ngrok.set_auth_token("Your Token")
public_url = ngrok.connect(8000)
print("FastAPI Public URL:", public_url)

nest_asyncio.apply()
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_app, daemon=True)
thread.start()

FastAPI Public URL: NgrokTunnel: "https://grenadierly-nonspinose-mollie.ngrok-free.dev" -> "http://localhost:8000"
